In [5]:
# Importation des librairies de base
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

import wrds
from scipy import stats
from scipy.stats import skew, kurtosis
import warnings

In [ ]:
# Connexion à la base de données WRDS
db = wrds.Connection(wrds_username='vince1209')

# Changer le nom après wrds_username pour te connecter, 
# entrer le nom d'utilisateur puis ton mot de passe dans la barre du haut, 
# puis "y" pour ne pas avoir à te reconnecter à chaque fois!

# Question 3

### Partie A

In [ ]:
# Importation des librairies supplémentaires
from scipy import stats
from scipy.stats import skew, kurtosis
import warnings
warnings.filterwarnings('ignore')

# Paramètres de simulation
np.random.seed(42)  # Pour reproductibilité
n_obs = 10000  # Nombre d'observations

print(f"🎯 Paramètres de simulation:")
print(f"  • Nombre d'observations: {n_obs}")
print(f"  • Graine aléatoire: 42 (pour reproductibilité)")

# 1. Simulation de la distribution gaussienne standard
print(f"\n📈 1. Simulation distribution gaussienne standard...")
returns_normal = np.random.normal(loc=0, scale=1, size=n_obs)
print(f"✅ {len(returns_normal)} observations gaussiennes générées")

# 2. Simulation de la distribution Student-t (3 degrés de liberté)
print(f"\n📈 2. Simulation distribution Student-t (df=3)...")
returns_t = stats.t.rvs(df=3, size=n_obs)
print(f"✅ {len(returns_t)} observations Student-t générées")

# 3. Normalisation pour même variance inconditionnelle
print(f"\n🔧 3. Normalisation des variances...")
print(f"Variance avant normalisation:")
print(f"  • Gaussienne: {np.var(returns_normal):.4f}")
print(f"  • Student-t: {np.var(returns_t):.4f}")

# Normaliser Student-t pour avoir la même variance que la gaussienne
target_variance = 1.0  # Variance de la gaussienne standard
returns_t_normalized = returns_t * np.sqrt(target_variance / np.var(returns_t))

print(f"Variance après normalisation:")
print(f"  • Gaussienne: {np.var(returns_normal):.4f}")
print(f"  • Student-t normalisée: {np.var(returns_t_normalized):.4f}")

# 4. Calcul des statistiques sommaires
print(f"\n📊 4. Statistiques sommaires:")

def calc_summary_stats(data, name):
    """Calcule les statistiques sommaires d'une série"""
    stats_dict = {
        'Moyenne': np.mean(data),
        'Écart-type': np.std(data),
        'Asymétrie (Skewness)': skew(data),
        'Aplatissement (Kurtosis)': kurtosis(data, fisher=True),  # Excès de kurtosis
        'Minimum': np.min(data),
        'Maximum': np.max(data)
    }
    
    print(f"\n{name}:")
    for stat_name, value in stats_dict.items():
        print(f"  • {stat_name}: {value:.4f}")
    
    return stats_dict

# Statistiques pour les deux distributions
stats_normal = calc_summary_stats(returns_normal, "Distribution Gaussienne")
stats_t = calc_summary_stats(returns_t_normalized, "Distribution Student-t (df=3, normalisée)")

In [ ]:
# 5. Visualisation des distributions
print(f"\n📈 5. Visualisation comparative des distributions...")

# Configuration des graphiques
plt.style.use('default')
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Comparaison: Distribution Gaussienne vs Student-t (df=3)', fontsize=16, y=0.98)

# Graphique 1: Histogrammes superposés
axes[0, 0].hist(returns_normal, bins=50, alpha=0.7, label='Gaussienne', color='blue', density=True)
axes[0, 0].hist(returns_t_normalized, bins=50, alpha=0.7, label='Student-t (df=3)', color='red', density=True)
axes[0, 0].set_title('Histogrammes superposés')
axes[0, 0].set_xlabel('Rendements')
axes[0, 0].set_ylabel('Densité')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Graphique 2: Estimation de densité par noyau (KDE)
from scipy.stats import gaussian_kde

kde_normal = gaussian_kde(returns_normal)
kde_t = gaussian_kde(returns_t_normalized)

x_range = np.linspace(-6, 6, 1000)
axes[0, 1].plot(x_range, kde_normal(x_range), label='Gaussienne', color='blue', linewidth=2)
axes[0, 1].plot(x_range, kde_t(x_range), label='Student-t (df=3)', color='red', linewidth=2)
axes[0, 1].set_title('Estimation de densité par noyau (KDE)')
axes[0, 1].set_xlabel('Rendements')
axes[0, 1].set_ylabel('Densité')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Graphique 3: Q-Q plots pour évaluer la normalité
from scipy.stats import probplot

probplot(returns_normal, dist="norm", plot=axes[1, 0])
axes[1, 0].set_title('Q-Q Plot: Gaussienne vs Normale théorique')
axes[1, 0].grid(True, alpha=0.3)

probplot(returns_t_normalized, dist="norm", plot=axes[1, 1])
axes[1, 1].set_title('Q-Q Plot: Student-t vs Normale théorique')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 6. Analyse des queues de distribution
print(f"\n🔍 6. Analyse détaillée des queues de distribution:")

# Calcul des quantiles extrêmes
quantiles = [0.01, 0.05, 0.95, 0.99]
print(f"\nQuantiles de distribution:")
print(f"{'Quantile':<10} {'Gaussienne':<12} {'Student-t':<12} {'Différence':<12}")
print("-" * 50)

for q in quantiles:
    q_normal = np.quantile(returns_normal, q)
    q_t = np.quantile(returns_t_normalized, q)
    diff = abs(q_t) - abs(q_normal)  # Différence en valeur absolue
    
    print(f"{q:<10.2f} {q_normal:<12.4f} {q_t:<12.4f} {diff:<12.4f}")

# Analyse des valeurs extrêmes
print(f"\n📊 Analyse des événements extrêmes:")
threshold_2sigma = 2.0  # Seuil à 2 écarts-types
threshold_3sigma = 3.0  # Seuil à 3 écarts-types

extreme_normal_2s = np.sum(np.abs(returns_normal) > threshold_2sigma)
extreme_t_2s = np.sum(np.abs(returns_t_normalized) > threshold_2sigma)

extreme_normal_3s = np.sum(np.abs(returns_normal) > threshold_3sigma)
extreme_t_3s = np.sum(np.abs(returns_t_normalized) > threshold_3sigma)

print(f"Événements |rendement| > 2σ:")
print(f"  • Gaussienne: {extreme_normal_2s} ({extreme_normal_2s/n_obs*100:.2f}%)")
print(f"  • Student-t: {extreme_t_2s} ({extreme_t_2s/n_obs*100:.2f}%)")
print(f"  • Théorique (normale): {0.0455*100:.2f}%")

print(f"\nÉvénements |rendement| > 3σ:")
print(f"  • Gaussienne: {extreme_normal_3s} ({extreme_normal_3s/n_obs*100:.2f}%)")
print(f"  • Student-t: {extreme_t_3s} ({extreme_t_3s/n_obs*100:.2f}%)")
print(f"  • Théorique (normale): {0.0027*100:.2f}%")

In [ ]:
# 7. Interprétation et discussion des résultats
print(f"\n📝 7. INTERPRÉTATION ET DISCUSSION:")
print("="*60)

print(f"\n🔍 ÉCARTS PAR RAPPORT À LA NORMALITÉ:")

print(f"\n1. ASYMÉTRIE (Skewness):")
skew_normal = skew(returns_normal)
skew_t = skew(returns_t_normalized)
print(f"   • Gaussienne: {skew_normal:.4f} (proche de 0 = symétrique)")
print(f"   • Student-t: {skew_t:.4f}")
if abs(skew_t) > 0.5:
    print(f"   ⚠️  Student-t montre une asymétrie notable")
else:
    print(f"   ✅ Les deux distributions sont approximativement symétriques")

print(f"\n2. APLATISSEMENT (Kurtosis excess):")
kurt_normal = kurtosis(returns_normal, fisher=True)
kurt_t = kurtosis(returns_t_normalized, fisher=True)
print(f"   • Gaussienne: {kurt_normal:.4f} (proche de 0 = normale)")
print(f"   • Student-t: {kurt_t:.4f}")
print(f"   📊 Différence de kurtosis: {kurt_t - kurt_normal:.4f}")

if kurt_t > 1:
    print(f"   ⚠️  Student-t présente un EXCÈS DE KURTOSIS significatif")
    print(f"   📈 Cela indique des QUEUES PLUS ÉPAISSES que la normale")
else:
    print(f"   ✅ Kurtosis proche de la normale")

print(f"\n3. QUEUES DE DISTRIBUTION:")
print(f"   📊 La distribution Student-t (df=3) est connue pour avoir des queues plus épaisses")
print(f"   📈 Probabilité d'événements extrêmes plus élevée que la gaussienne")
print(f"   ⚠️  Sous-estimation du risque si on assume la normalité pour des données Student-t")

print(f"\n4. IMPLICATIONS POUR LE RISQUE FINANCIER:")
print(f"   🏦 VaR Gaussienne sous-estimera le risque réel si les données suivent Student-t")
print(f"   📉 Les pertes extrêmes seront plus fréquentes que prévu")
print(f"   🎯 Importance de tester les hypothèses distributionnelles")

print(f"\n5. TESTS DE NORMALITÉ RECOMMANDÉS:")
# Test de Jarque-Bera
from scipy.stats import jarque_bera

jb_normal, p_normal = jarque_bera(returns_normal)
jb_t, p_t = jarque_bera(returns_t_normalized)

print(f"   Test de Jarque-Bera (H0: normalité):")
print(f"   • Gaussienne: JB = {jb_normal:.4f}, p-value = {p_normal:.6f}")
print(f"   • Student-t: JB = {jb_t:.4f}, p-value = {p_t:.6f}")

if p_normal > 0.05:
    print(f"   ✅ Gaussienne: On ne rejette pas H0 (normalité acceptée)")
else:
    print(f"   ❌ Gaussienne: Rejet de H0 (non-normalité)")

if p_t > 0.05:
    print(f"   ✅ Student-t: On ne rejette pas H0 (normalité acceptée)")
else:
    print(f"   ❌ Student-t: Rejet de H0 (non-normalité détectée)")

print(f"\n🎯 CONCLUSION PARTIE A:")
print(f"   La distribution Student-t (df=3) présente des caractéristiques distinctes:")
print(f"   • Queues plus épaisses (kurtosis élevée)")
print(f"   • Plus d'événements extrêmes")
print(f"   • Risque sous-estimé par les modèles gaussiens")
print(f"   • Nécessité de modèles de VaR robustes aux queues épaisses")

print(f"\n✅ Simulation et analyse terminées!")